In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random


In [ ]:
response = requests.get("https://gamesdonequick.com/tracker/donations/agdq2020?page=3")

In [ ]:
soup = BeautifulSoup(response.text, "lxml")

In [ ]:
all_tr = soup.find_all('tr')
len(all_tr)

In [ ]:
all_tr[25]

In [ ]:
all_tr[25].find_all('td')

In [ ]:
all_tr[25].find_all('td')[0].text

In [ ]:
all_tr[1].find_all('td')[2].a

In [ ]:
all_tr[1].find_all('a')

In [ ]:
all_tr[1].find_all('td')[-1].text.strip()

In [ ]:
base_url = 'https://gamesdonequick.com/tracker/donations/agdq2020?page='
name = []
name_url = []
time_received = []
amount = []
amount_url = []
comment = []

# this is to loop through the pages
for i in range(1, 1083):
    print(i)
    # set up the url
    url = base_url + str(i)
    response = requests.get(url)
    
    # structure the html into a BeautifulSoup object
    soup = BeautifulSoup(response.text, "lxml")
    
    # find all tr items
    all_tr = soup.find_all('tr')
    
    # loop through the tr table rows
    for tr in all_tr[1:]:
        
        # get a breakdown of all the items
        td_list = tr.find_all('td')
        
        # append the data
        name.append(td_list[0].text.strip())
        
        # in case of anonymous
        try:
            name_url.append(td_list[0].a['href'])
        except:
            name_url.append(None)
        
        # time
        time_received.append(td_list[1].span.text)
        
        # amount
        amount.append(td_list[2].a.text.strip())
        amount_url.append(td_list[2].a['href'])
        
        #comment
        comment.append(td_list[3].text.strip())

# build dataframe
aqdq_df = pd.DataFrame({"name": name,
              "name_url": name_url,
              "time_received": time_received,
              "amount": amount,
              "amount_url": amount_url,
              "comment": comment})

In [ ]:
aqdq_df.head()

In [ ]:
aqdq_df.tail()

In [ ]:
aqdq_df.isnull().sum()

In [ ]:
aqdq_df.to_csv("adqd_df_2020_01_13.csv", index=None)

## Checkpoint

In [2]:
aqdq_df = pd.read_csv("adqd_df_2020_01_13.csv")

aqdq_df.head()

,name,name_url,time_received,amount,amount_url,comment
0,ducomors,/tracker/donor/434740/28,01/12/2020 16:19:07 +0000,$5.00,/tracker/donation/665700,No
1,(No Name),/tracker/donor/434739/28,01/12/2020 16:06:30 +0000,$25.00,/tracker/donation/665699,No
2,Mitchel,/tracker/donor/434738/28,01/12/2020 14:17:00 +0000,$25.00,/tracker/donation/665698,Yes
3,mellyn,/tracker/donor/336894/28,01/12/2020 14:14:38 +0000,$25.00,/tracker/donation/665697,No
4,Joemoe224,/tracker/donor/371976/28,01/12/2020 13:42:53 +0000,$25.00,/tracker/donation/665696,No


In [3]:
aqdq_df['comment'].value_counts()

Yes    36592
No     17508
Name: comment, dtype: int64

In [4]:
amount_float = [float(i.replace("$","").replace(",", "")) for i in aqdq_df['amount']]

aqdq_df['amount_float'] = amount_float

aqdq_df.head()

,name,name_url,time_received,amount,amount_url,comment,amount_float
0,ducomors,/tracker/donor/434740/28,01/12/2020 16:19:07 +0000,$5.00,/tracker/donation/665700,No,5.0
1,(No Name),/tracker/donor/434739/28,01/12/2020 16:06:30 +0000,$25.00,/tracker/donation/665699,No,25.0
2,Mitchel,/tracker/donor/434738/28,01/12/2020 14:17:00 +0000,$25.00,/tracker/donation/665698,Yes,25.0
3,mellyn,/tracker/donor/336894/28,01/12/2020 14:14:38 +0000,$25.00,/tracker/donation/665697,No,25.0
4,Joemoe224,/tracker/donor/371976/28,01/12/2020 13:42:53 +0000,$25.00,/tracker/donation/665696,No,25.0


In [ ]:
base_donation_url = "https://gamesdonequick.com/"

demo_url = "/tracker/donation/665698"

url = base_donation_url + demo_url

donation_response = requests.get(url)

donation_soup = BeautifulSoup(donation_response.text, 'lxml')

donation_soup

In [ ]:
donation_soup.find_all('td')

In [ ]:
base_donation_url = "https://gamesdonequick.com/"
comment_list = []

start = time.time()

for index, row in aqdq_df[:10000].iterrows():
    if row['comment'] == "Yes":
        try:
            donation_url = row['amount_url']
            url = base_donation_url + donation_url
            donation_response = requests.get(url)
            donation_soup = BeautifulSoup(donation_response.text, "lxml")
            comment_td = donation_soup.find_all('td')[0]
            #print(comment_td.text.strip())
            comment_list.append(comment_td.text.strip())
        except:
            comment_list.append(None)
    else:
        comment_list.append(None)

end = time.time()
print("")
print("Time Taken: {:.6f}s".format(end-start))

In [ ]:
comment_list.count(None)

In [5]:
def retrieveComment(comment_url):
    base_donation_url = "https://gamesdonequick.com/"
    url = base_donation_url + comment_url
    donation_response = requests.get(url)
    donation_soup = BeautifulSoup(donation_response.text, "lxml")
    #print(donation_soup)
    comment_td = donation_soup.find_all('td')[0]
    #print(comment_td.text.strip())
    temp_df = pd.DataFrame({"amount_url": [comment_url], "comment": [comment_td.text.strip()]})
    time.sleep(random.randint(1,4))
    return temp_df

In [ ]:
for i in aqdq_df[:10]['amount_url']:
    print(retrieveComment(i))

In [7]:
amount_url_df = aqdq_df[aqdq_df['comment'] == "Yes"]
amount_url_df = amount_url_df.reset_index(drop=True)
amount_url_list = amount_url_df['amount_url']

In [40]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

with ThreadPoolExecutor(max_workers=3) as executor:
    start = time.time()
    donation_results = []
    # 0 - 1000
    # 1000 - 5000
    # 5000 - 10000
    # 10000 - 15000
    # 15000 - 20000
    # 20000 - 35000
    # 35000 - end
    futures = [executor.submit(retrieveComment, comment_url) for comment_url in amount_url_list[35000:]]
    for result in as_completed(futures):
        donation_results.append(result)
    end = time.time()
    print("Time Taken: {:.6f}s".format(end-start))

Time Taken: 1699.959710s


In [41]:
donation_results

[<Future at 0xb72590f7b8 state=finished returned DataFrame>,
 <Future at 0xb727267898 state=finished returned DataFrame>,
 <Future at 0xb7259194e0 state=finished returned DataFrame>,
 <Future at 0xb727267198 state=finished returned DataFrame>,
 <Future at 0xb727267128 state=finished returned DataFrame>,
 <Future at 0xb727267048 state=finished returned DataFrame>,
 <Future at 0xb722de9080 state=finished returned DataFrame>,
 <Future at 0xb71f74b208 state=finished returned DataFrame>,
 <Future at 0xb722de94e0 state=finished returned DataFrame>,
 <Future at 0xb722de9a20 state=finished returned DataFrame>,
 <Future at 0xb722de9940 state=finished returned DataFrame>,
 <Future at 0xb725b069b0 state=finished returned DataFrame>,
 <Future at 0xb722de9320 state=finished returned DataFrame>,
 <Future at 0xb725b06a58 state=finished returned DataFrame>,
 <Future at 0xb725b06cf8 state=finished returned DataFrame>,
 <Future at 0xb725b06978 state=finished returned DataFrame>,
 <Future at 0xb725b06fd0

In [42]:
donation_results[-10:]

[<Future at 0xb724c88b00 state=finished returned DataFrame>,
 <Future at 0xb724c885c0 state=finished returned DataFrame>,
 <Future at 0xb724c88d30 state=finished returned DataFrame>,
 <Future at 0xb724c88240 state=finished returned DataFrame>,
 <Future at 0xb724c88208 state=finished returned DataFrame>,
 <Future at 0xb724c53ac8 state=finished returned DataFrame>,
 <Future at 0xb724c53c88 state=finished returned DataFrame>,
 <Future at 0xb724c53f60 state=finished returned DataFrame>,
 <Future at 0xb724c53a90 state=finished returned DataFrame>,
 <Future at 0xb724c53518 state=finished returned DataFrame>]

In [43]:
donation_35000_end = []

for i in donation_results:
    try:
        donation_35000_end.append(i.result())
    except:
        pass
    
donation_35000_end_combined = pd.concat(donation_35000_end)
donation_35000_end_combined.head()

,amount_url,comment
0,/tracker/donation/604970,Train
0,/tracker/donation/604969,$5 55:55 HYPE TRAIN!!!!
0,/tracker/donation/604971,$5 at 55:55
0,/tracker/donation/604967,(Comment rejected)
0,/tracker/donation/604966,5 dollars for 55:55 on Super Mario Bros. 3!


In [44]:
donation_35000_end_combined.tail()

,amount_url,comment
0,/tracker/donation/602246,"Well, after an intro like that, I can't NOT do..."
0,/tracker/donation/602245,Pre-show and hat hype!
0,/tracker/donation/602243,uwu :3
0,/tracker/donation/602240,LFG LFG!
0,/tracker/donation/602237,Milguh!


In [45]:
donation_35000_end_combined.to_csv("donation_comment_35000_end.csv", index=None)